In [1]:
import pandas as pd
import time
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd

In [8]:
query = "SELECT  ano,  tipo_vinculo,  sigla_uf,  sum(quantidade_vinculo) as quantidade_vinculos FROM  `basedosdados.br_ibge_estadic.indicadores_quantidade_vinculo` where ano =2021 group by   1, 2, 3"

In [3]:
client = bigquery.Client()

In [9]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████| 135/135 [00:00<00:00, 355.36rows/s]


In [10]:
y=df.pivot_table(index={'sigla_uf'},values='quantidade_vinculos', aggfunc=np.sum)

In [11]:
y = pd.DataFrame(y)
total = y.reset_index()


In [12]:
total=total.rename(columns={'quantidade_vinculos':'total_estado'})

In [13]:
df1 = df.merge(total, right_on='sigla_uf', left_on='sigla_uf')

In [14]:
## Subindo para GBQ

dataset_ref = client.dataset('perfil_remuneracao') 

In [29]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 0 to 134
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ano                  135 non-null    Int64 
 1   tipo_vinculo         135 non-null    object
 2   sigla_uf             135 non-null    object
 3   quantidade_vinculos  132 non-null    Int64 
 4   total_estado         135 non-null    Int64 
dtypes: Int64(3), object(2)
memory usage: 6.7+ KB


In [15]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
bigquery.SchemaField('tipo_vinculo','STRING',description='Tipo de vinculo.'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('total_estado','INTEGER',description='Total de vínculos daquela UF')
 ]

In [16]:
table_ref = dataset_ref.table('ESTADIC_quantidade_vinculos_mapa') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=7d82a612-996f-4c28-b214-8a50d3c71741>